In [2]:
import pandas as pd

### Задание 1
Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце 'region' пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим 'undefined'.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {

    'Центр': ['москва', 'тула', 'ярославль'],

    'Северо-Запад': ['петербург', 'псков', 'мурманск'],

    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']

}

Результат классификации запишите в отдельный столбец region.

In [28]:
df = pd.read_csv('keywords.csv')

In [29]:
#Загружаем соответсвие городов через конфигурационный файл
from yaml import load, dump
config = load( open('geo_data.yaml', encoding = 'utf8') )
config

{'geo_data': {'Центр': ['москва', 'тула', 'ярославль'],
  'Северо-Запад': ['петербург', 'псков', 'мурманск'],
  'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']}}

In [34]:
def define_region(row):
    '''Проставляет соответствие между городами и регионами в запросе
    Если нашлось более одного соответствия, регион оставляем неопределенным'''
    geo_data = config['geo_data']
    keywords = row['keyword'].split()
    possible_regions = [key for key in geo_data if set(geo_data[key]) & set(keywords)]
    return possible_regions[0] if len(possible_regions) == 1 else 'undefined'

In [35]:
df['region'] = df.apply(define_region, axis=1)

In [47]:
df[df['region'] != 'undefined'].head()

,keyword,shows,region
127,авито москва,979292,Центр
370,авито ру санкт петербург,425134,Северо-Запад
849,авито ярославль,209581,Центр
1063,фарпост владивосток,176951,Дальний Восток
1236,банк санкт петербург,174375,Северо-Запад


### Задание 2
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
    - оценка 2 и меньше - низкий рейтинг
    - оценка 4 и меньше - средний рейтинг
    - оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [40]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100004 entries, 0 to 100003
Data columns (total 4 columns):
userId       100004 non-null int64
movieId      100004 non-null int64
rating       100004 non-null float64
timestamp    100004 non-null int64
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [45]:
def define_rating(row):
    '''Производит классификацию по рейтингам'''
    return 'high' if row['rating'] > 4.5 else 'middile' if  row['rating'] > 2 else 'low'

In [43]:
ratings['class'] = ratings.apply(define_rating, axis=1)

In [46]:
ratings.head()

,userId,movieId,rating,timestamp,class
0,1,31,2.5,1260759144,middile
1,1,1029,3.0,1260759179,middile
2,1,1061,3.0,1260759182,middile
3,1,1129,2.0,1260759185,low
4,1,1172,4.0,1260759205,middile


### Задание 3
Посчитайте среднее значение Lifetime киноманов (пользователи, которые поставили 100 и более рейтингов). Под Lifetime понимается разница между максимальным и минимальным значением timestamp для каждого пользователя. Ответ дайте в днях.

In [3]:
#загружаем датасет
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [4]:
#определяем киноманов
rat_group = ratings.groupby('userId').count()
rat_group_filtered = rat_group[rat_group['movieId'] > 100].reset_index()
rat_group_filtered.head()

,userId,movieId,rating,timestamp
0,4,204,204,204
1,8,116,116,116
2,15,1700,1700,1700
3,17,363,363,363
4,19,423,423,423


In [5]:
#фильтруем исходный датасет по id киноманов
usr_lst = rat_group_filtered['userId'].tolist()

In [6]:
ratin_with_usr = ratings[ratings['userId'].isin(usr_lst)]

In [28]:
#находим минимальное и максимальное timestamp для отфильтрованных юзеров. переводи в привычный формат даты
x  = ratin_with_usr.groupby('userId').agg(['min','max'])
x['day_max'] = x['timestamp']['max'].apply(datetime.fromtimestamp)
x['day_min'] = x['timestamp']['min'].apply(datetime.fromtimestamp)

In [43]:
#находим разность между минимальным и максимальным значением
x['delta'] = (x['day_max'] - x['day_min'])
x.sort_values(by='delta',ascending=False).head(10)

movieId         rating        timestamp              \
           min     max    min  max         min         max   
userId                                                       
380          1  140928    0.5  5.0   949367125  1465156786   
547          1  163949    0.5  5.0   974777109  1476587644   
427        246  115174    0.5  5.0   939332587  1415901843   
15           1  161155    0.5  5.0   997937239  1469330735   
624          1  161918    0.5  5.0  1019123866  1476616373   
529          1  112460    0.5  5.0   959963968  1408296622   
648          6  155611    1.0  5.0  1138477436  1460757570   
388          6   60756    1.0  5.0   946508346  1229027041   
311          1   45208    0.5  5.0   897403673  1147824793   
472          1   50068    1.0  5.0   939002838  1172956897   

                   day_max             day_min              delta  
                                                                   
userId                                                             
380    2016-06-05 22:59:46 2000-02-01 04:05:25 5969 days 18:54:21  
547    2016-10-16 06:14:04 2000-11-21 06:25:09 5807 days 23:48:55  
427    2014-11-13 21:04:03 1999-10-08 00:43:07 5515 days 20:20:56  
15     2016-07-24 06:25:35 2001-08-16 07:47:19 5455 days 22:38:16  
624    2016-10-16 14:12:53 2002-04-18 12:57:46 5295 days 01:15:07  
529    2014-08-17 20:30:22 2000-06-02 19:39:28 5189 days 00:50:54  
648    2016-04-16 00:59:30 2006-01-28 22:43:56 3730 days 02:15:34  
388    2008-12-11 23:24:01 1999-12-30 01:59:06 3269 days 21:24:55  
311    2006-05-17 03:13:13 1998-06-09 17:47:53 2898 days 09:25:20  
472    2007-03-04 00:21:37 1999-10-04 05:07:18 2707 days 19:14:19

In [47]:
#находим среднее разности. ответ возвращаем в днях
x['delta'].mean().days

463

### Задание 4
Есть мнение, что "раньше снимали настоящее кино, не то что сейчас". Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из материалов занятия. Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

1. В переменную years запишите список из всех годов с 1950 по 2010.

2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:
    - для каждой строки пройдите по всем годам списка years
    - если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
    - если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год

3. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец 'year'

4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца 'year' и отсортируйте результат по убыванию рейтинга

In [52]:
#загружаем датасеты
ratings = pd.read_csv('ml-latest-small/ratings.csv')
movies = pd.read_csv('ml-latest-small/movies.csv')

In [54]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [55]:
#создаем список лет с 1950 по 2010
years = [str(x) for x in range(1950, 2011)]

In [56]:
#определяем функцию для идентификации года в названии
def define_year(row):
    for x in years:
        if x in row['title']:
            return x
    return '1900'

In [57]:
#применяем функцию к датасету movies
movies['year'] = movies.apply(define_year, axis = 1)

In [58]:
movies.head()

,movieId,title,genres,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II (1995),Comedy,1995


In [59]:
#джоиним датасеты movies и ratings
joined = ratings.merge(movies, on='movieId', how='left')
joined.head()

,userId,movieId,rating,timestamp,title,genres,year
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama,1995
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical,1900
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller,1996
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller,1981
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama,1989


In [62]:
#находим средние рейтинги по годам
joined.groupby('year')['rating'].mean().sort_values(ascending=False).head(20)

year
1957    4.014241
1972    4.011136
1952    4.000000
1954    3.994220
1951    3.983539
1974    3.978704
1962    3.952446
1950    3.915254
1977    3.905786
1964    3.841492
1959    3.841033
1958    3.823171
1975    3.823077
1969    3.817376
1967    3.814516
1960    3.809211
1976    3.806794
1973    3.806667
1979    3.777966
1981    3.758763
Name: rating, dtype: float64